## Working with fundamental data


#### Libraries

In [9]:
import pandas as pd
import numpy as np
import os

#### Reading income statements

In [10]:
files = os.listdir('data/IncomeStatement/')
period = 'annual' # 'quarterly' # 'annual'

tickers = ['AAPL']

for t in tickers: 
    for f in files:   # Oops...
        if t in f and period in f:  
            filename = f
            print(f)

AAPL-Income-Statement-annual-24-02-2021-17-23.csv


#### Auxiliar function

This function creates a DF from the IncomeStatement, then it uses it to create a table with all the tickers and its income data.

In [11]:
def create_df_ticker(ticker_name, Statement_type, filename):
    """Given the name of the .csv file it returns a dataframe for a single ticker."""    
    # Read file
    income_ticker = pd.read_csv('data/' + Statement_type + '/' + filename, skiprows = 1)
    
    # Rename column
    income_ticker.columns = ['Data'] + list(income_ticker.columns[1:])
    # income_ticker.rename(columns = {'Fiscal year ends in September. USD in millions except per share data.': 'Data'}, inplace = True)
    
    # Fixing duplicated data names
    eps_index = income_ticker[income_ticker.iloc[:,0] == 'Earnings per share'].index[0]
    income_ticker.iloc[eps_index + 1, 0] += ' ' + income_ticker.iloc[eps_index, 0]
    income_ticker.iloc[eps_index + 2, 0] += ' ' + income_ticker.iloc[eps_index, 0]
    #income_ticker = pd.concat([income_ticker.iloc[:eps_index], income_ticker.iloc[eps_index+1:]])

    waso_index = income_ticker[income_ticker.iloc[:,0] == 'Weighted average shares outstanding'].index[0]
    income_ticker.iloc[waso_index + 1, 0] += ' ' + income_ticker.iloc[waso_index, 0]
    income_ticker.iloc[waso_index + 2, 0] += ' ' + income_ticker.iloc[waso_index, 0]
    #income_ticker = pd.concat([income_ticker.iloc[:waso_index], income_ticker.iloc[waso_index+1:]])
    income_ticker = income_ticker.set_index('Data').drop(['Operating expenses', 'Earnings per share', 'Weighted average shares outstanding'])

     # Change column names to eliminate month (we only care about years)
    income_ticker.columns = [x[:4] for x in income_ticker.columns]

    initial_year = int(income_ticker.columns[0])
    years_to_process = [str(initial_year + x) for x in range(1, 5)]  #skip first one...
    
    for x in ['Revenue', 'Cost of revenue', 'Gross profit', 'Income before taxes', 'Diluted Earnings per share',
              'Net income available to common shareholders']:
        
        if x not in income_ticker.index:
            income_ticker.loc[x] = np.nan  # Add full of NaN
        
        income_ticker.loc['Change % YoY {}'.format(x)] = np.nan
        for y in years_to_process:
            income_ticker.loc['Change % YoY' + x, y] = (income_ticker.loc[x, y] / income_ticker.loc[x, str(int(y)-1)] - 1) * 100
    
        
    income_ticker.reset_index(inplace = True)
    
    # Transposing
    income_ticker = income_ticker.transpose()
    income_ticker.reset_index(inplace = True)

    # Renaming columns as the first row
    income_ticker.columns = income_ticker.iloc[0].values

    # Dropping first row
    income_ticker = income_ticker.iloc[1:]
    
    # Creating 'Year' and 'Ticker' columns
    income_ticker['Year'] = income_ticker['Data'].str[:4]
    income_ticker['Ticker'] = ticker_name
    
    # Setting 'Ticker' as index, dropping and sorting columns
    #income_ticker.set_index('Ticker', inplace = True)
    income_ticker.drop(['Data'], axis = 1, inplace = True)
    income_ticker = income_ticker.iloc[:, [income_ticker.shape[1] - 1, income_ticker.shape[1] - 2] + list(range(income_ticker.shape[1] - 2))  ]
    
    return income_ticker

In [12]:
create_df_ticker('DE', 'IncomeStatement', 'DE-Income-Statement-annual-24-02-2021-18-10.csv')

,Ticker,Year,Revenue,Cost of revenue,Gross profit,Research and development,"Sales, General and administrative",Other operating expenses,Total operating expenses,Operating income,...,Change % YoY Cost of revenue,Change % YoYCost of revenue,Change % YoY Gross profit,Change % YoYGross profit,Change % YoY Income before taxes,Change % YoYIncome before taxes,Change % YoY Diluted Earnings per share,Change % YoYDiluted Earnings per share,Change % YoY Net income available to common shareholders,Change % YoYNet income available to common shareholders
1,DE,2016,26364.0,19175.0,7189.0,1389.0,2952.0,640.0,4981.0,2208.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE,2017,29116.0,21001.0,8115.0,1368.0,3254.0,732.0,5353.0,2761.0,...,NaN,9.522816,NaN,12.88079,NaN,41.816547,NaN,38.877339,NaN,41.694025
3,DE,2018,37021.0,26986.0,10036.0,1658.0,3630.0,691.0,5979.0,4056.0,...,NaN,28.498643,NaN,23.672212,NaN,29.074192,NaN,8.383234,NaN,9.731233
4,DE,2019,38941.0,28486.0,10455.0,1783.0,3694.0,915.0,6392.0,4063.0,...,NaN,5.558438,NaN,4.17497,NaN,0.417588,NaN,40.19337,NaN,37.373311
5,DE,2020,35259.0,25112.0,10147.0,1644.0,3677.0,969.0,6290.0,3857.0,...,NaN,-11.844415,NaN,-2.945959,NaN,-5.014677,NaN,-14.384236,NaN,-15.431909
6,DE,TTM,35259.0,25112.0,10147.0,1644.0,3677.0,969.0,6290.0,3857.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Main loop

Putting together IncomeStatement of all tickers

In [13]:
files = os.listdir('data/IncomeStatement/')

tickers_downloaded = set()

for f in files:
    tickers_downloaded.add(f.split(' ')[0].split('-')[0])
    
tickers_downloaded = sorted(list(tickers_downloaded))

In [14]:
# Descarga la información...
# ... PENDING ...

# Procesa la información
Statement_type = 'IncomeStatement'
period = 'annual' # 'quarterly' # 'annual'

# Listing all files
files = os.listdir('data/' + Statement_type + '/')

Fundamentals = pd.DataFrame()

for t in tickers_downloaded:    # tickers
    # Look for file
    for f in files:  
        if t in f and period in f:  
            filename = f
    
#    print(filename)
    df_ticker = create_df_ticker(t, Statement_type, filename)
    
    if Fundamentals.shape == (0,0):
        Fundamentals = df_ticker
    else:
        Fundamentals = pd.concat([Fundamentals, df_ticker])

Fundamentals.head(50)

<ipython-input-11-91193cecf54b>:36: RuntimeWarning: invalid value encountered in double_scalars
  income_ticker.loc['Change % YoY' + x, y] = (income_ticker.loc[x, y] / income_ticker.loc[x, str(int(y)-1)] - 1) * 100
<ipython-input-11-91193cecf54b>:36: RuntimeWarning: divide by zero encountered in double_scalars
  income_ticker.loc['Change % YoY' + x, y] = (income_ticker.loc[x, y] / income_ticker.loc[x, str(int(y)-1)] - 1) * 100


,Ticker,Year,Revenue,Cost of revenue,Gross profit,Research and development,"Sales, General and administrative","Restructuring, merger and acquisition",Other operating expenses,Total operating expenses,...,Change % YoYIncome before taxes,Change % YoY Diluted Earnings per share,Change % YoYDiluted Earnings per share,Change % YoY Net income available to common shareholders,Change % YoYNet income available to common shareholders,Other,Preferred dividend,Net income from discontinuing ops,Other income,Extraordinary items
1,A,2016,3574.0,1932.0,1642.0,376.0,751.0,144.0,85.0,1356.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2017,3722.0,2012.0,1710.0,389.0,749.0,66.0,118.0,1322.0,...,-168.217054,NaN,-112.075472,NaN,-112.408759,NaN,NaN,NaN,NaN,NaN
3,A,2018,4218.0,2237.0,1981.0,444.0,811.0,19.0,78.0,1352.0,...,495.454545,NaN,2325.0,NaN,2376.470588,NaN,NaN,NaN,NaN,NaN
4,A,2019,4485.0,2385.0,2100.0,447.0,826.0,32.0,71.0,1376.0,...,14.122137,NaN,28.479381,NaN,29.216152,NaN,NaN,NaN,NaN,NaN
5,A,2020,4448.0,2445.0,2003.0,453.0,787.0,34.0,44.0,1318.0,...,-6.354515,NaN,-6.218656,NaN,-7.352941,NaN,NaN,NaN,NaN,NaN
6,A,TTM,4448.0,2445.0,2003.0,453.0,787.0,34.0,44.0,1318.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,2016,40180.0,28339.0,11841.0,NaN,1323.0,NaN,4525.0,5848.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAL,2017,42207.0,31154.0,11053.0,NaN,1477.0,505.0,4481.0,6463.0,...,-28.262387,NaN,-18.918919,NaN,-28.28849,NaN,NaN,NaN,NaN,NaN
3,AAL,2018,44541.0,34490.0,10051.0,NaN,1520.0,748.0,4384.0,6652.0,...,-38.910506,NaN,-22.307692,NaN,-26.42001,NaN,NaN,NaN,NaN,NaN
4,AAL,2019,45768.0,35379.0,10389.0,NaN,1602.0,473.0,4627.0,6702.0,...,19.745223,NaN,25.082508,NaN,19.405099,NaN,NaN,NaN,NaN,NaN


In [15]:
sorted(Fundamentals.columns)

['Basic Earnings per share',
 'Basic Weighted average shares outstanding',
 'Change % YoY Cost of revenue',
 'Change % YoY Diluted Earnings per share',
 'Change % YoY Gross profit',
 'Change % YoY Income before taxes',
 'Change % YoY Net income available to common shareholders',
 'Change % YoY Revenue',
 'Change % YoYCost of revenue',
 'Change % YoYDiluted Earnings per share',
 'Change % YoYGross profit',
 'Change % YoYIncome before taxes',
 'Change % YoYNet income available to common shareholders',
 'Change % YoYRevenue',
 'Cost of revenue',
 'Diluted Earnings per share',
 'Diluted Weighted average shares outstanding',
 'EBITDA',
 'Extraordinary items',
 'Gross profit',
 'Income before taxes',
 'Interest Expense',
 'Net income',
 'Net income available to common shareholders',
 'Net income from continuing operations',
 'Net income from discontinuing ops',
 'Operating income',
 'Other',
 'Other income',
 'Other income (expense)',
 'Other operating expenses',
 'Preferred dividend',
 'Pro